# Homework 6

Author: Xuyuan Zhang; Time: Feb. 1. 2024

Notebook with practicing and Ch 14 Create your own example

This is a problem I encounter in codewars, and I provide my solution and the origianl source of this problem

## [Metaclasses - Simple Django Models](https://www.codewars.com/kata/54b26b130786c9f7ed000555)

**DESCRIPTION**:

Django is a famous back-end framework written in Python. It has a vast list of features including the creation of database tables through "models". You can see an example of such model below:

```
class Person(models.Model):
    first_name = models.CharField()
    last_name = models.CharField()
```

Apart from creating a table it can perform validation, generate HTML forms, and so on. This is possible thanks to metaclasses. Normally there are better solutions than using metaclasses, but they can be of great help in creating powerful framework interfaces. This goal of this kata is to learn and understand how such frameworks works.

Your task is to implement a class Model and classes for its fields to support functionality like in the following example:

```
class User(Model):
    first_name = CharField(max_length=30)
    last_name = CharField(max_length=50)
    email = EmailField()
    is_verified = BooleanField(default=False)
    date_joined = DateTimeField(auto_now=True)
    age = IntegerField(min_value=5, max_value=120, blank=True)


user1 = User(first_name='Liam', last_name='Smith', email='liam@example.com')
user1.validate()

print(user1.date_joined)  # prints date and time when the instance was created
print(user1.is_verified)  # prints False (default value)

user1.age = 256
user1.validate()  # raises ValidationError - age is out of range

user2 = User()
user2.validate()  # raises ValidationError - first three fields are missing and mandatory
```

The classes which inherit from Model should:

* support creation of fields using class-attribute syntax 
* have a validate method which checks whether all fields are valid

The field types you should implement are described below. Each of them also has parameters blank (default False), which determines whether None is allowed as a value, and default (default None) which determines the value to be used if nothing was provided at instantiation time of the Model.

* CharField - a string with min_length (default 0) and max_length (default None) parameters, both inclusive if defined
* IntegerField - an integer with min_value (default None) and max_value (default None) parameters, both inclusive if defined
* BooleanField - a boolean
* DateTimeField - a datetime with an extra parameter auto_now (default False). If auto_now is True and no default value has been provided, the current datetime should be used automatically at Model instantion time.
* EmailField - a string in the format of address@subdomain.domain where address, subdomain, and domain are sequences of alphabetical characters with min_length (default 0) and max_length (default None) parameters

Each field type should have its own validate method which checks whether the provided value has the correct type and satisfies the length/value constraints.

In [6]:
import datetime
from typing import Any
import re

class ValidationError(Exception):
    def __init__(self, message):
        super().__init__(message)
    

class ValidationField:
    def __init__(self, default = None, blank = False, **kwargs):
        self.default = default
        self.blank = blank
        self.__dict__.update(kwargs)

    def validate(self, validation):
        if validation is None and self.blank is False:
            raise ValidationError('Field cannot be blank')
        
class CharField(ValidationField):
    def __init__(self, min_length = 0, max_length = None, **kwargs):
        super().__init__(**kwargs) # use parent class
        self.min_length = min_length
        self.max_length = max_length

    def validate(self, validation):
        super().validate(validation)
        
        if validation is None:
            return None
        
        if not isinstance(validation, str):
            raise ValidationError('Field must be a string')
        
        if self.min_length is not None and len(validation) < self.min_length:
            raise ValidationError('Field must be at least {} characters long'.format(self.min_length))
        
        if self.max_length is not None and len(validation) > self.max_length:
            raise ValidationError('Field must be at most {} characters long'.format(self.max_length))

class IntegerField(ValidationField):
    def __init__(self, min_value = None, max_value = None, **kwargs):
        super().__init__(**kwargs)
        self.min_value = min_value
        self.max_value = max_value
    
    def validate(self, validation):
        super().validate(validation)
        if validation is None:
            return None
        if isinstance(validation, bool):
            raise ValidationError('Field must be an integer')
        if not isinstance(validation, int):
            raise ValidationError('Field must be an integer')
        if self.min_value is not None and validation < self.min_value:
            raise ValidationError('Age is out of range')
        if self.max_value is not None and self.max_value < validation:
            raise ValidationError('Age is out of range')

class BooleanField(ValidationField):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
    
    def validate(self, validation):
        super().validate(validation)
        
        if validation is None:
            return None
        
        if not isinstance(validation, bool):
            raise ValidationError('Field must be a boolean')
        
class DateTimeField(ValidationField):
    def __init__(self, default=None, auto_now=False, **kwargs):
        super().__init__(**kwargs)
        self.auto_now = auto_now
        self._default = default

    def __getattribute__(self, name: str) -> Any:
        if name == 'default':
            if self._default is None and self.auto_now is True:
                return datetime.datetime.now()
            else:
                return self._default
        return super().__getattribute__(name)
    
    def validate(self, validation):
        super().validate(validation)
        
        if validation is None:
            return None
        
        if not isinstance(validation, datetime.datetime):
            raise ValidationError('Field must be a date')
        
class EmailField(ValidationField):
    def __init__(self, min_length=0, max_length=None, **kwargs):
        super().__init__(**kwargs)
        self.min_length = min_length
        self.max_length = max_length
        
    def validate(self, validation):
        super().validate(validation)
        
        if validation is None:
            return  # just return without null
        
        if not isinstance(validation, str):
            raise ValidationError('Field must be a string')
        
        if self.min_length is not None and len(validation) < self.min_length:
            raise ValidationError('Field must be at least {} characters long'.format(self.min_length))
    
        if self.max_length is not None and len(validation) > self.max_length:
            raise ValidationError('Field must be at most {} characters long'.format(self.max_length))
        
        if not re.match(r"[^@]+@[^@]+\.[^@]+", validation):
            raise ValidationError("Wrong email format")
        
class Model:
    def __init__(self, **kwargs):
        # Initialize fields with defaults
        for field_name, attr in self.fields().items():
            setattr(self, field_name, kwargs.get(field_name, attr.default))

    def __init_subclass__(cls) -> None:
        # Prepare fields in subclass
        fields = {name: getattr(cls, name) for name in dir(cls) if isinstance(getattr(cls, name), ValidationField)}
        for name, field in fields.items():
            setattr(cls, '_f_' + name, field)
            delattr(cls, name)
        super().__init_subclass__()

    @classmethod
    def fields(cls):
        # Return fields of the class
        return {name[3:]: attr for name, attr in cls.__dict__.items() if name.startswith('_f_')}

    def validate(self):
        # Validate all fields
        for field_name, field in self.fields().items():
            field.validate(getattr(self, field_name))


my solution is passed and the following is the voted solution (approximately good solution)

In [ ]:
import datetime
import re


class ValidationError(Exception):
    pass


class Field:
    def __init__(self, default=None, blank=False):
        self.name = ''
        self._default = default
        self.blank = blank

    @property
    def default(self):
        if callable(self._default):
            return self._default()
        return self._default

    def validate(self, value):
        if not self.blank and value is None:
            raise ValidationError(self.name, 'missing value')

        if value is not None and not self.is_type_ok(value):
            raise ValidationError(self.name, 'wrong type')

    def is_type_ok(self, value):
        return True


class CharField(Field):
    def __init__(self, min_length=0, max_length=None, **kwds):
        super(CharField, self).__init__(**kwds)
        self.min_length = min_length
        self.max_length = max_length
    
    def validate(self, value):
        super(CharField, self).validate(value)
        
        if value is not None and self.min_length and len(value) < self.min_length:
            raise ValidationError(self.name, 'too short')
        
        if value is not None and self.max_length and len(value) > self.max_length:
            raise ValidationError(self.name, 'too long')
    
    def is_type_ok(self, value):
        return isinstance(value, str)


class EmailField(CharField):
    def validate(self, value):
        super(EmailField, self).validate(value)
        
        if value is not None and not re.match(r'[.a-z]+@[a-z]+\.[a-z]{2,6}', value):
            raise ValidationError(self.name, 'not valid e-mail')


class BooleanField(Field):
    def is_type_ok(self, value):
        return type(value) == bool


class DateTimeField(Field):
    def __init__(self, auto_now=False, **kwds):
        if auto_now and kwds.get('default') is None:
            kwds['default'] = datetime.datetime.now
        super(DateTimeField, self).__init__(**kwds)
        self.auto_now = auto_now
    
    def is_type_ok(self, value):
        return isinstance(value, datetime.datetime)


class IntegerField(Field):
    def __init__(self, min_value=None, max_value=None, **kwds):
        super(IntegerField, self).__init__(**kwds)
        self.min_value = min_value
        self.max_value = max_value
    
    def validate(self, value):
        super(IntegerField, self).validate(value)
        
        if value is not None and self.min_value and value < self.min_value:
            raise ValidationError(self.name, 'too small')

        if value is not None and self.max_value and value > self.max_value:
            raise ValidationError(self.name, 'too big')
    
    def is_type_ok(self, value):
        return type(value) == int


class ModelMeta(type):
    def __new__(meta, class_name, bases, class_dict):
        new_class_dict = {}
        
        for attribute_name, attribute in class_dict.items():
            if not isinstance(attribute, Field):
                new_class_dict[attribute_name] = attribute
                continue
            attribute.name = attribute_name
            new_class_dict.setdefault('_attributes_', {}).setdefault(attribute_name, attribute)
        
        return super(ModelMeta, meta).__new__(meta, class_name, bases, new_class_dict)


class Model(metaclass=ModelMeta):
    _attributes_ = {}
    
    def __init__(self, **kwds):
        for attr in self._attributes_.values():
            setattr(self, attr.name, kwds.get(attr.name, attr.default))
    
    def validate(self):
        for attr in self._attributes_.values():
            attr.validate(getattr(self, attr.name))